# **IEEE GRSS Coastal Erosion Monitoring**
Deep learning based coastal erosion rate prediction model

In [10]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from torch.utils.data import DataLoader, TensorDataset
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# **Dataset setup + defining variables**

In [3]:
#dataset..
df = pd.read_excel("cedt.xlsx", header=None)

df.columns = ["year", "country", "continent", "population", "area",
              "erosion_rate", "lat", "lon", "elevation"]

print("✅ Data loaded successfully.")
print(df.head(), "\n")

✅ Data loaded successfully.
   year        country continent  population  area  erosion_rate         lat  \
0  1950        lesotho    Africa       30360     0     13.706192 -342.568355   
1  1950        liberia    Africa       96320   842     25.376183 -342.568355   
2  1950          libya    Africa     1759540  2000     22.509892 -342.568355   
3  1950  liechtenstein    Europe         160     0      4.842375 -342.568355   
4  1950      lithuania    Europe       62674   258      6.123858 -342.568355   

         lon  elevation  
0 -80.700729     0.0000  
1 -80.700729    13.6920  
2 -80.700729     5.1829  
3 -80.700729     0.0000  
4 -80.700729     0.2300   



In [6]:
# setting column
y = df["erosion_rate"]
X = df.drop(columns=["erosion_rate"])

# categorical and numerical columns
categorical_cols = ["country", "continent"]
numerical_cols = [col for col in X.columns if col not in categorical_cols]

# Combine preprocessing steps
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numerical_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
])

# Fit + transform data
X_processed = preprocessor.fit_transform(X)

# Convert to torch tensors
X_tensor = torch.tensor(X_processed.toarray() if hasattr(X_processed, "toarray") else X_processed, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=32)

# **Pytorch Model class**

In [7]:
#Define Neural Network
class ErosionModel(nn.Module):
    def __init__(self, input_dim):
        super(ErosionModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

# Instantiate model
model = ErosionModel(X_train.shape[1])

# **Training phase**

In [8]:
#Setting up for training
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 100


# Training Loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {total_loss/len(train_dl):.4f}")

#Saving model
torch.save(model.state_dict(), "erosion_model_weights.pth")
joblib.dump(preprocessor, "erosion_preprocessor.pkl")

print("\n💾 Model weights saved as 'erosion_model_weights.pth'")
print("💾 Preprocessor saved as 'erosion_preprocessor.pkl'")


Epoch [10/100] - Loss: 7.2665
Epoch [20/100] - Loss: 0.5874
Epoch [30/100] - Loss: 0.2304
Epoch [40/100] - Loss: 0.1547
Epoch [50/100] - Loss: 0.1221
Epoch [60/100] - Loss: 0.0973
Epoch [70/100] - Loss: 0.0820
Epoch [80/100] - Loss: 0.0731
Epoch [90/100] - Loss: 0.0694
Epoch [100/100] - Loss: 0.0728

💾 Model weights saved as 'erosion_model_weights.pth'
💾 Preprocessor saved as 'erosion_preprocessor.pkl'


# **Evaluation Phase**

In [11]:
#Evaluation Phase

model.eval()
with torch.no_grad():
    preds = model(X_test)
    test_loss = criterion(preds, y_test)
    print(f"\n✅ Test MSE: {test_loss.item():.4f}")



✅ Test MSE: 0.2246


In [13]:
model.eval()
with torch.no_grad():
    preds = model(X_test)
    y_true = y_test.numpy()
    y_pred = preds.numpy()

# Calculate metrics
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print("\n📊 Evaluation Metrics:")
print(f"MSE  (Mean Squared Error): {mse:.4f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f"MAE  (Mean Absolute Error): {mae:.4f}")
print(f"R² Score (Model Accuracy): {r2:.4f}")


📊 Evaluation Metrics:
MSE  (Mean Squared Error): 0.2246
RMSE (Root Mean Squared Error): 0.4739
MAE  (Mean Absolute Error): 0.3050
R² Score (Model Accuracy): 0.9967
